In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from notion_client import Client
from time import sleep


In [ ]:
# Load environment variables from .env file
load_dotenv()

# Access the secrets
secret_key = os.getenv('SECRET_KEY')
database_url = os.getenv('DATABASE_URL')

print(f'Secret Key: {secret_key}')
print(f'Database URL: {database_url}')

notion = Client(auth=os.environ["NOTION_API_TOKEN"])


In [ ]:
def get_job_desc(page_id):
    blocks = notion.blocks.children.list(page_id)
    lines = []
    for b in blocks["results"]:
        # Type can be e.g. 'paragraph' or 'bulleted_list_item'
        b_type = b["type"]
        # If line is not empty
        if len(b[b_type]["rich_text"]) != 0:
            lines.append(b[b_type]["rich_text"][0]["text"]["content"])
    return "\n".join(lines)

def get_gpt_query(job_desc):
    # Import prompt examples
    with open("gpt-prompt-examples.txt", "r") as f:
        job_desc_examples = f.read()

    gpt_query = f"""
    Write 1 paragraph of max 100 words for the CV Description, keeping the EXACT SAME structure as the examples.
    Mention the underlined keywords, incuding tasks (e.g. "Apache Spark", "Streamlit") and technologies (e.g. "Microsoft Azure") ONLY if they were mentioned in the new job desciption.

    ### NEW Job description.
    {job_desc}

    {job_desc_examples}
    """
    return gpt_query
    

# Example usage
job_desc = get_job_desc("4469a74a8198491a854c6cbb8728979b")
gpt_query = get_gpt_query(job_desc)

# Test the query on ChatGPT
with open("gpt-prompt-output.txt", "w") as f:
    f.write(gpt_query)

# Query OpenAI API

In [ ]:
client = OpenAI()

def get_gpt_response(GPT_QUERY):
  completion = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
      {"role": "system", "content": "You are an expert in writing CV, specialized in writing CVs for with only very technical words."},
      {"role": "user", "content": GPT_QUERY}
    ]
  )

  print("Query cost:", completion.usage)

  gpt_response = completion.choices[0].message.content
  return gpt_response

## Write description in Notion field

## Unofficial library

In [ ]:
# Problem: I can't get the values from a database, only from a page + the callback does
# not work.

# from notionpy.notion.client import NotionClient

# # Obtain the `token_v2` value by inspecting your browser cookies on a logged-in (non-guest) session on Notion.so
# # client = NotionClient(token_v2=os.environ["NOTION_CLIENT_TOKEN"])
# client = NotionClient(token_v2="v02%3Auser_token_or_cookies%3AIuGhcIACfb6pVlM8VUq7-9X-l1X-Iw9OIY8TUsZ52hQtYu3moRx4P3USsEufcMz4uoJO-HqFiLn9B1g5uZqxX8wFSgWDKdlsW0Tw4eKk-5zKy5SsGqetGGrSfDzxLfseHg7f")

# # Replace this URL with the URL of the page you want to edit
# # page = client.get_block("https://www.notion.so/marcodifrancesco/Data-Intern-Analyst-4469a74a8198491a854c6cbb8728979b")
# page = client.get_block("https://www.notion.so/marcodifrancesco/8993d6d12560487fbfa2ebd3e93962ad?v=f10b880e1c0f472981705bfbbcb066b6")

# print("The title is:", page.title)

# # Get new job description
# # Exclude empty lines
# job_desc = "\n".join([p.title for p in page.children if p.title != ""])
# job_desc


# while True:
#     from time import sleep
#     sleep(1)
#     page.refresh()

## Official library

In [ ]:
def get_db(db_id):
    db = notion.databases.query(
        **{
            "database_id": db_id,
            # "filter": {
            #     "property": "Landmark",
            #     "rich_text": {
            #         "contains": "Bridge",
            #     },
            # },
        }
    )
    return db

def set_to_processing(page_id):
    update_properties = {
        "properties": {
            "_Req Desc Prog": {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {
                            "content": "🟡 Processing..."
                        }
                    }
                ]
            }
        }
    }

    response = notion.pages.update(
        page_id=page_id,
        **update_properties
    )

def set_descr_in_notion(page_id, gpt_response):
    # Set Description, and change status to "Done"
    update_properties = {
        "properties": {
            "CV Description": {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {
                            "content": gpt_response
                        }
                    }
                ]
            },
            "_Req Desc Prog": {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {
                            "content": "🟢 Done. Click button 'Req Desc' to request again."
                        }
                    }
                ]
            },
        }
    }

    response = notion.pages.update(
        page_id=page_id,
        **update_properties
    )

def update_db(db):
    for job_post in db["results"]:
        change_req = job_post["properties"]["_Req Desc Prog"]["rich_text"]    
        if len(change_req) != 0:
            assert len(change_req) == 1, "Unhandled case"
            if change_req[0]["text"]["content"] == "🔵 Requested…":
                set_to_processing(job_post["id"])
                job_desc = get_job_desc(job_post["id"])
                gpt_query = get_gpt_query(job_desc)
                gpt_response = get_gpt_response(gpt_query)
                set_descr_in_notion(job_post["id"], gpt_response)
                print(f"Updated {job_post["id"]}")

In [ ]:
def main():
    while True:
        db = get_db("8993d6d12560487fbfa2ebd3e93962ad")
        update_db(db)
        sleep(2)
        print("DB checked")


main()